In [314]:
from pybiomart import Dataset
from pandas_plink import read_plink
import pyranges as pr
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [315]:
dataset = Dataset(name='btaurus_gene_ensembl',
                  host='http://www.ensembl.org')

for attribute in dataset.attributes.keys():
    if 'reactome' in attribute:
        print(attribute)

reactome
reactome_gene
reactome_transcript


In [316]:
results = dataset.query(attributes=[
    'ensembl_gene_id',
    'external_gene_name',
    'go_id',
    'reactome'
])

In [317]:
gen_path = '../data/ML/BBB2023_MD'
bim, fam, bed = read_plink(gen_path)

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  2.27it/s]


In [318]:
snps_pos = bim[['snp','chrom', 'pos']]
# counts = snps_pos.groupby('snp').cumcount()
# snps_pos.loc[counts > 0, 'snp'] = snps_pos.loc[counts > 0, 'snp'] + '_' + counts.loc[counts > 0].astype(str)

snps_pos.rename(columns={'snp':'title', 'pos': 'End', 'chrom': 'Chromosome'}, inplace=True)
snps_pos['Start'] = snps_pos['End']-1

# snps_gr = pr.PyRanges(
#     chromosomes = snps_pos["chrom"],
#     starts = snps_pos["pos"] -1,
#     ends=snps_pos["pos"],
# )

snps_gr = pr.PyRanges(snps_pos)
snps_gr

/tmp/ipykernel_123646/3430236937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_pos.rename(columns={'snp':'title', 'pos': 'End', 'chrom': 'Chromosome'}, inplace=True)
/tmp/ipykernel_123646/3430236937.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_pos['Start'] = snps_pos['End']-1


,title,Chromosome,End,Start
0,BTA-101873,1,776231,776230
1,ARS-BFGL-NGS-16466,1,907810,907809
2,ARS-BFGL-NGS-105096,1,993060,993059
3,BES1_Contig627_1906,1,1032564,1032563
4,ARS-BFGL-NGS-98142,1,1110393,1110392
...,...,...,...,...
36299,ARS-BFGL-NGS-16401,29,50416855,50416854
36300,ARS-BFGL-NGS-3284,29,50522805,50522804
36301,BTA-66351,29,50548328,50548327
36302,ARS-BFGL-NGS-91027,29,50595638,50595637


In [ ]:
genes = pr.read_gtf("../data/Bos_taurus.UMD3.1.90.gtf")

genes = genes[genes.Feature == "gene"]

overlap = snps_gr.join(genes)

In [ ]:
overlap = overlap[['title','Chromosome', 'Start', 'End', 'Source', 'Feature', 'Start_b', 'End_b',
       'Score', 'Strand', 'Frame', 'gene_id', 'gene_version', 'gene_name',
       'gene_source', 'gene_biotype']]

print(overlap.columns)

Index(['title', 'Chromosome', 'End', 'Start', 'Source', 'Feature', 'Start_b',
       'End_b', 'Score', 'Strand', 'Frame', 'gene_id', 'gene_version',
       'gene_name', 'gene_source', 'gene_biotype'],
      dtype='object')


In [ ]:
overlap.df.head()

,title,Chromosome,End,Start,Source,Feature,Start_b,End_b,Score,Strand,Frame,gene_id,gene_version,gene_name,gene_source,gene_biotype
0,ARS-BFGL-NGS-66449,1,1204825,1204824,ensembl,gene,1189227,1222846,.,+,.,ENSBTAG00000008490,5,CRYZL1,ensembl,protein_coding
1,Hapmap53766-ss46526150,1,2082139,2082138,ensembl,gene,2067035,2099875,.,+,.,ENSBTAG00000003064,5,PAXBP1,ensembl,protein_coding
2,ARS-BFGL-NGS-62826,1,2158691,2158690,ensembl,gene,2109218,2196114,.,+,.,ENSBTAG00000003063,5,SYNJ1,ensembl,protein_coding
3,ARS-BFGL-NGS-39992,1,2391777,2391776,ensembl,gene,2382872,2459950,.,+,.,ENSBTAG00000012412,4,URB1,ensembl,protein_coding
4,BTB-00001612,1,3492221,3492220,ensembl,gene,3483532,3721930,.,+,.,ENSBTAG00000017839,5,TIAM1,ensembl,protein_coding


In [ ]:
from gprofiler.gprofiler import GProfiler

gp = GProfiler(return_dataframe=True)

# List of unique gene symbols
gene_list = overlap.df['gene_name'].dropna().unique().tolist()

# Query for pathways
gp = GProfiler(return_dataframe=True)

results = gp.profile(
    organism='btaurus',
    query=gene_list,
    sources=['KEGG', 'GO:BP', 'REAC'],  # KEGG, GO biological process, Reactome
    no_evidences=False
)

# View results
print(results.head())

  source      native                                name       p_value  \
0  GO:BP  GO:0065007               biological regulation  7.361622e-80   
1  GO:BP  GO:0050789    regulation of biological process  3.232581e-71   
2  GO:BP  GO:0048856    anatomical structure development  2.762056e-69   
3  GO:BP  GO:0032502               developmental process  9.587048e-67   
4  GO:BP  GO:0007275  multicellular organism development  2.003839e-60   

   significant                                        description  term_size  \
0         True  "Any process that modulates a measurable attri...       8335   
1         True  "Any process that modulates the frequency, rat...       8113   
2         True  "The biological process whose specific outcome...       3832   
3         True  "A biological process whose specific outcome i...       4098   
4         True  "The biological process whose specific outcome...       3013   

   query_size  intersection_size  effective_domain_size  precision    reca

In [ ]:
results['native']

0      GO:0065007
1      GO:0050789
2      GO:0048856
3      GO:0032502
4      GO:0007275
          ...    
586    GO:0021915
587    GO:0021954
588    GO:1903131
589    GO:0007611
590    GO:0090130
Name: native, Length: 591, dtype: object

### Constructing the neural network using information from the genes and pathways on the structure

In [ ]:
df = pd.DataFrame()
counts = overlap.df.groupby('title').cumcount()
overlap.df['title'] = overlap.df['title'] + '_' +counts.astype(str)

In [ ]:
# snp_list = overlap.df['title'].tolist()
# gene_list = overlap.df['gene_name'].unique().tolist()

# n_snps = len(set(snp_list))
# n_genes = len(gene_list)

# # Map SNP/Gene to index
# snp_idx = {}
# for i, snp in enumerate(snp_list):
#     if snp in snp_idx.keys():
#         snp_idx[snp] = snp_idx[snp]+[i]
#     else:
#         snp_idx[snp] = [i]

# gene_idx = {gene:i for i,gene in enumerate(gene_list)}

# # Initialize mask
# mask_snp_gene = np.zeros((n_genes, n_snps), dtype=np.float32)

# for _, row in overlap.df.iterrows():
#     i = gene_idx[row['gene_name']]
#     for j in snp_idx[row['title']]:
#         mask_snp_gene[i,j] = 1.0

In [ ]:
snp_genes = overlap.df[['title', 'gene_id']]

snp_genes_unique = (
    snp_genes
    .groupby('title', as_index=False)['gene_id']
    .agg(list)
)

print(f"The snp_genes_unique has shape {snp_genes_unique.shape}")

snp_list = snp_genes_unique['title'].tolist()
gene_list = overlap.df['gene_id'].unique().tolist()

print(f"The snp_list has {len(snp_list)} size")

n_snps = len(snp_list)
n_genes = len(gene_list)

print(f"The n_snps has {n_snps} size")

snp_idx = {snp:i for i,snp in enumerate(snp_list)}
gene_idx = {gene:i for i,gene in enumerate(gene_list)}

print(f"The snp_idx has {len(snp_idx)}")

mask_snp_gene = np.zeros((n_genes, n_snps), dtype=np.float32)

for snp in snp_list:
    j = snp_idx[snp]
    for gene in snp_genes_unique.loc[snp_genes_unique['title'] == snp,'gene_id'].iloc[0]:
        i = gene_idx[gene]
        mask_snp_gene[i,j] = 1.0

print(f"The mask_snp_gene has shape {mask_snp_gene.shape}")

print(f"The sum of the mask_snp_gene is {mask_snp_gene.sum()}")

The snp_genes_unique has shape (11208, 2)
The snp_list has 11208 size
The n_snps has 11208 size
The snp_idx has 11208
The mask_snp_gene has shape (6055, 11208)
The sum of the mask_snp_gene is 11295.0


In [ ]:
# for i,_ in enumerate(mask_snp_gene.T):
#     if mask_snp_gene.T[i,:].sum() >1 :
#         print(f"{i} .... {mask_snp_gene.T[i,:].sum()}") 

288 .... 2.0
473 .... 2.0
489 .... 2.0
511 .... 2.0
597 .... 2.0
627 .... 2.0
871 .... 2.0
969 .... 2.0
1163 .... 2.0
1217 .... 2.0
1234 .... 2.0
1411 .... 2.0
1682 .... 2.0
1738 .... 3.0
1744 .... 2.0
1813 .... 2.0
1995 .... 2.0
2064 .... 2.0
2189 .... 2.0
2420 .... 2.0
2466 .... 2.0
2527 .... 2.0
2746 .... 2.0
2754 .... 2.0
2826 .... 2.0
2936 .... 2.0
2998 .... 2.0
3060 .... 2.0
3233 .... 2.0
3371 .... 2.0
3499 .... 2.0
3703 .... 2.0
3790 .... 2.0
3939 .... 2.0
4192 .... 2.0
4515 .... 2.0
4538 .... 2.0
4596 .... 2.0
4604 .... 2.0
4808 .... 2.0
4891 .... 2.0
5289 .... 2.0
5330 .... 2.0
5805 .... 2.0
5906 .... 2.0
6081 .... 2.0
6362 .... 2.0
6488 .... 2.0
6531 .... 2.0
6573 .... 2.0
6612 .... 2.0
6899 .... 2.0
7025 .... 2.0
7089 .... 2.0
7151 .... 2.0
7222 .... 2.0
7769 .... 2.0
7956 .... 2.0
8037 .... 2.0
8048 .... 2.0
8126 .... 2.0
8449 .... 2.0
8616 .... 2.0
8823 .... 2.0
8871 .... 2.0
8880 .... 2.0
8893 .... 2.0
8894 .... 2.0
9013 .... 2.0
9014 .... 2.0
9293 .... 2.0
9300 .... 2.0


In [ ]:
# for snp in snp_idx.keys():
#     if snp_idx[snp] == 473:
#         print(snp)

ARS-BFGL-NGS-101891


In [ ]:
# snp_genes_unique[snp_genes_unique['title'] == 'ARS-BFGL-NGS-101891']

,title,gene_id
473,ARS-BFGL-NGS-101891,"[ENSBTAG00000010577, ENSBTAG00000010571]"


In [ ]:
overlap.df.loc[ overlap.df['gene_name'] == "APP", 'gene_id'].iloc[0]

'ENSBTAG00000017753'

In [ ]:
gene_names = overlap.df[['gene_name', 'gene_id']]

In [ ]:
# # Defining pathways
# pathway_list = results['native'].unique()

# n_pathway = len(pathway_list)

# # Mapping pathway to index
# idx_pathway = {pathway:i for i,pathway in enumerate(pathway_list)}

# mask_gene_pathway = np.zeros((n_pathway, n_genes), dtype=np.float32)

# for _,row in results.iterrows():
#     genes_intersec = row['intersections']
#     i = idx_pathway[row['native']]
#     for gene in genes_intersec:
#         gene_id = gene_names[ gene_names['gene_name'] == gene, 'gene_id'].iloc[0]
#         j = gene_idx[gene_id]
#         mask_gene_pathway[i,j] = 1.0
    

In [ ]:
# Defining pathways
pathway_list = results['native'].unique()
n_pathway = len(pathway_list)

idx_pathway = {pathway: i for i, pathway in enumerate(pathway_list)}

gene_name_to_id = dict(zip(gene_names['gene_name'], gene_names['gene_id']))

mask_gene_pathway = np.zeros((n_pathway, n_genes), dtype=np.float32)

for row in results.itertuples(index=False):
    i = idx_pathway[row.native]

    for gene in row.intersections:
        gene_id = gene_name_to_id.get(gene)
        if gene_id is None:
            continue
        
        j = gene_idx.get(gene_id)
        if j is not None:
            mask_gene_pathway[i, j] = 1.0

In [ ]:
print(mask_gene_pathway.shape)
print(mask_gene_pathway.sum())

(591, 6055)
135740.0


In [ ]:
class MaskedLinear(nn.Module):
    def __init__(self, in_features, out_features, mask):
        super().__init__()

        self.weight = nn.Parameter(torch.randn(out_features, in_features)*0.01)
        self.bias = nn.Parameter(torch.zeros(out_features))

        self.register_buffer("mask", mask)

    def forward(self, x):
        masked_weight = self.weight * self.mask
        return F.linear(x, masked_weight, self.bias)
    
class PartialNet(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2, fc1_dim, fc2_dim, mask1, mask2):
        super().__init__()

        # Masked layers
        self.masked1 = MaskedLinear(input_dim, hidden1, mask1)
        self.masked2 = MaskedLinear(hidden1, hidden2, mask2)

        # FC layers
        self.fc1 = nn.Linear(hidden2, fc1_dim)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc3 = nn.Linear(fc2_dim, 1)

        # Activation
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.masked1(x))
        x = self.activation(self.masked2(x))
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [ ]:
# Selectioning the correct SNPs

bim = bim.set_index('snp')
snp_used = snp_genes_unique.set_index('title')
snp_used.index.name = 'snp'
snp_to_index = bim.loc[snp_used.index]['i']

# Getting the matrix indexed correctly
X = bed[snp_to_index]#.compute()


KeyError: "None of ['snp'] are in the columns"

In [ ]:
pheno_df = pd.read_csv("../data/ML/pheno_2023bbb_0twins_6traits_mask", delimiter="\t", header=None)

pheno_df = pheno_df.drop(columns=[0])
pheno_df = pheno_df.set_index(1)
pheno_df.index.name == "iid"

# Separating data to training
y = pheno_df[2][~pheno_df[2].isna()].to_numpy()
X = X.T[~pheno_df[2].isna()].compute()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 11208 but corresponding boolean dimension is 18324

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1000, random_state=42)

scaler_X = StandardScaler()

X_train = scaler_X.fit_transform(X_train)   # fit ONLY on train
X_test  = scaler_X.transform(X_test) 

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

scaler_y = StandardScaler()

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test  = scaler_y.transform(y_test.reshape(-1, 1))

y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test  = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
X_train.shape

(13330, 11208)

In [ ]:
input_dim = n_snps
hidden1 = n_genes
hidden2 = n_pathway
fc1_dim = 100
fc2_dim = 100
mask1 = torch.from_numpy(mask_snp_gene)
mask2 = torch.from_numpy(mask_gene_pathway)

In [ ]:
model = PartialNet(input_dim, hidden1, hidden2,
                   fc1_dim, fc2_dim,
                   mask1, mask2)

criterion = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 100

In [ ]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(torch.from_numpy(X_train).float())
    loss = criterion(outputs, torch.from_numpy(y_train).float().unsqueeze(1))

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

Epoch 0, Loss: 3.152222
Epoch 1, Loss: 3.152098
Epoch 2, Loss: 3.151975
Epoch 3, Loss: 3.151852
Epoch 4, Loss: 3.151728
Epoch 5, Loss: 3.151605
Epoch 6, Loss: 3.151481
Epoch 7, Loss: 3.151358
Epoch 8, Loss: 3.151235
Epoch 9, Loss: 3.151111
Epoch 10, Loss: 3.150988


KeyboardInterrupt: 